<a href="https://colab.research.google.com/github/0xJCarlos/QuestionAnswering/blob/main/QuestionAnswering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Primeramente instalamos las librerias necesarias

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
## Cargar dataset
from datasets import load_dataset
squad = load_dataset("squad", split="train[:5000]")

In [3]:
## Dividir el dataset en sets de train y test

squad = squad.train_test_split(test_size=0.2)

In [4]:
## Observar un ejemplo
squad["train"][0]

{'id': '56cfed4d234ae51400d9c0df',
 'title': 'New_York_City',
 'context': "New York City has the largest European and non-Hispanic white population of any American city. At 2.7 million in 2012, New York's non-Hispanic white population is larger than the non-Hispanic white populations of Los Angeles (1.1 million), Chicago (865,000), and Houston (550,000) combined. The European diaspora residing in the city is very diverse. According to 2012 Census estimates, there were roughly 560,000 Italian Americans, 385,000 Irish Americans, 253,000 German Americans, 223,000 Russian Americans, 201,000 Polish Americans, and 137,000 English Americans. Additionally, Greek and French Americans numbered 65,000 each, with those of Hungarian descent estimated at 60,000 people. Ukrainian and Scottish Americans numbered 55,000 and 35,000, respectively. People identifying ancestry from Spain numbered 30,838 total in 2010. People of Norwegian and Swedish descent both stood at about 20,000 each, while people of 

In [5]:
## Cargar DistilBERT tokenizer para procesar los campos "question" y "context"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [6]:
### HAYALGUNOS PASOS DE PREPROCESAMIENTO PARA QUESTION-ANSWERING DE LOS QUE SE DEBERÍA ESTÁR CONSCIENTES:

#1. Some examples in a dataset may have a very long context that exceeds the maximum input length of the model.
#To deal with longer sequences, truncate only the context by setting truncation="only_second"

#2. Next, map the start and end positions of the answer to the original context by setting
# return_offset_mapping=True

#3. With the mapping in hand, now you can find the start and end tokens of the answer.
#Use the sequence_ids method to find which part of the offset corresponds to the question and which corresponds to the context.

In [7]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs


In [8]:
## Usar la funcion map para aplicar la función de preprocesamiento en todo el dataset. Puedes acelerarlo usando batched=True
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [9]:
### Crear un batch de ejemplos usando DefaultDataCollator. No aplica ningún preprocesamiento adicional como padding
from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors="tf")

## Entrenar el modelo!

In [10]:
from transformers import  create_optimizer

batch_size = 16
num_epochs = 2
total_train_steps = (len(tokenized_squad["train"]) // batch_size) * num_epochs
optimizer, schedule = create_optimizer(
    init_lr = 2e-5,
    num_warmup_steps = 0,
    num_train_steps = total_train_steps,
)

In [11]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [12]:
#Convertir dataset a formato tf.data.Dataset con prepare_tf_dataset():
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle = True,
    batch_size = 16,
    collate_fn = data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["test"],
    shuffle = False,
    batch_size = 16,
    collate_fn = data_collator,
)

In [13]:
#Configura el modelo para entrenamiento con compile
import tensorflow as tf
model.compile(optimizer=optimizer)

In [14]:
from transformers.keras_callbacks import PushToHubCallback

callback = PushToHubCallback(
    output_dir="my_awesome_qa_model",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
/content/my_awesome_qa_model is already a clone of https://huggingface.co/0xJCarlos/my_awesome_qa_model. Make sure you pull the latest changes with `repo.git_pull()`.


In [15]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=[callback])

Epoch 1/3
250/250 [==============================] - 231s 843ms/step - loss: 3.5919 - val_loss: 2.3177
Epoch 2/3
250/250 [==============================] - 218s 873ms/step - loss: 1.9745 - val_loss: 1.8597
Epoch 3/3
250/250 [==============================] - 209s 837ms/step - loss: 1.6940 - val_loss: 1.8597


In [25]:
question = "What is La Michoacana?"
context = "La Michoacana is a group of different Mexican ice cream parlors, with an estimated 8 to 15 thousand locations in Mexico."

In [26]:
from transformers import pipeline

question_answerer = pipeline("question-answering", model="my_awesome_qa_model")

question_answerer(question=question, context=context)

Some layers from the model checkpoint at my_awesome_qa_model were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForQuestionAnswering were not initialized from the model checkpoint at my_awesome_qa_model and are newly initialized: ['dropout_119']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'score': 0.08526315540075302,
 'start': 38,
 'end': 63,
 'answer': 'Mexican ice cream parlors'}